# BlueFors Temperature Control Setup

In [21]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import h5py
import inspect
from tqdm import tqdm
import sys
import math
import presto
from presto import lockin, utils, hardware
from presto.hardware import AdcFSample, AdcMode, DacFSample, DacMode
from blueftc.BlueforsController import BlueFTController

# Reload credentials module to get latest changes
import importlib
import JPA_credentials
importlib.reload(JPA_credentials)
from JPA_credentials import API_KEY, IP_ADDRESS, PORT_NUMBER, MXC_ID, HEATER_ID, PID_CALIB_FILE

# Timeout for hardware communication
import requests

_real_get = requests.get
def get_with_timeout(*args, **kwargs):
    kwargs.setdefault("timeout", (3.0, 5.0))  # (connect_timeout, read_timeout)
    return _real_get(*args, **kwargs)

requests.get = get_with_timeout

## `READ` Commands

- Reading the temperature of each channel (1 to 8)
- Reading the resistance of each channel (1 to 8)
- Reading the status of the mixing chamber heater

In [ ]:
controller = BlueFTController(
	ip=IP_ADDRESS, 						# Laptop IP address
	port=PORT_NUMBER, 					# Port number from Bluefors Temperature Control interface	
	key=API_KEY, 						# API key generated with explicit permisions from Bluefors Temperature Control interface								
	mixing_chamber_channel_id=MXC_ID, 
	mixing_chamber_heater_id=HEATER_ID, 
	pid_calib_path=PID_CALIB_FILE		# Path to PID calibration file - Need to calibrate manually first
	)

active_channels = [
	1, 				# 50K stage
	2, 				# 4K stage
	5, 				# Still
	6  				# Mixing Chamber
]

# Channel temperatures and resistances
for ch in active_channels:
	print(f"Channel {ch} temp: {controller.get_channel_temperature(ch)} Kelvin")
	print(f"Channel {ch} resistance: {controller.get_channel_resistance(ch)} Ohm")

# Mixing chamber heater status
print(f"MXC heater status: {controller.get_mxc_heater_status()}")
print(f"MXC heater power: {controller.get_mxc_heater_power()} uW")
print(f"MXC heater PID: {controller.get_mxc_heater_mode()}")
print(f"MXC heater setpoint: {controller.get_mxc_heater_setpoint()} K")
print(f"MXC heater PID config: {controller.get_mxc_heater_pid_config()}")

# `WRITE` Commands

These commands require an API key with read and write permission and can potentially cause 
substantial damage to the hardware. Only execute with caution and absolute certainty of 
what is going to happen!

All write commands return True if executed successfully, otherwise False.

In [ ]:
# turn the mixing chamber heater on or off
status = controller.toggle_mxc_heater('off')

# set power of the mixing chamber heater, in microwatts
status = controller.set_mxc_heater_power(100)

# set the temperature set point (mK) of the mixing chamber PID control
status = controller.set_mxc_heater_setpoint(30, use_pid=True)

# turn the mixing chamber PID control on (True) or off (False)
status = controller.set_mxc_heater_mode(True)

# Temperature Ramp Up

Module for step wise temperature ramp up of the mixing chamber from $10$ mK $\to$ $700$ mK. 

- Increase temperature set point $T_{SP}$ by $\Delta T$ (say, 10mK)

- Check temperature: `if` $T_{DR} \in T_{SP} \pm T_{error}$ (say, 0.1mK):  
    1. Wait for $\tau_{rest} = $ 1 minute for full thermalization
    2. Run script to sample voltages over bandwidth

In [ ]:
# Import Planck Spectrometry Script
# =================================
import JPA_planck_spec
importlib.reload(JPA_planck_spec)
from JPA_planck_spec import *

# Define Temperature Set Points for Data Acquisition
# ==================================================
temp_set_low = np.arange(10, 200, 10)  # unit: mK, up to 200mK in 10mK steps for fine resolution at low temps
temp_set_high = np.arange(200, 700, 50)  # unit: mK, from 200mK to 700mK in 50mK steps for faster sweep at higher temps
temp_set = np.concatenate((temp_set_low, temp_set_high))

# Run through Temperature Set Points and Acquire Data
# ===================================================
for temp in temp_set:
    print(f"\nSetting MXC temperature to {temp} mK...")
    controller.set_mxc_heater_setpoint(temp / 1000, use_pid=True)  # setpoint in K

    # Wait for temperature to stabilize
    stable = False
    while not stable:
        time.sleep(30)  # wait 30 seconds before checking temperature again
        current_temp = controller.get_channel_temperature(6) * 1000  # get current temp in mK
        print(f"Current MXC temperature: {current_temp:.2f} mK")
        if abs(current_temp - temp) < 0.1:  # check if within 0.1 mK of setpoint
            stable = True
            print(f"Temperature stabilized at {current_temp:.2f} mK")

    # Acquire Planck Spectrometry Data at Current Temperature
    print(f"Acquiring Planck spectrometry data at {temp} mK...")
    acquire_planck_data(temp)

print("\nData acquisition complete.")



